### 0.1. Install Dependencies

In [ ]:
# Install required packages
get_ipython().run_line_magic('pip', 'uninstall -qy google-genai')
get_ipython().run_line_magic('pip', 'install -qU "google-genai==1.7.0" "pandas" "pytrends"')

print("✅ Packages installed.")

### 0.2. Import Libraries

In [ ]:
import os
import json
import warnings

import pandas as pd
from google import genai
from google.genai import types as genai_types
from kaggle_secrets import UserSecretsClient
from IPython.display import Markdown, display
from pytrends.request import TrendReq

# Initialize PyTrends
try:
    pytrends_client = TrendReq(hl='en-US', tz=360)
    print("✅ PyTrends initialized.")
except Exception as e:
    print(f"⚠️ PyTrends init failed: {e}")
    pytrends_client = None

warnings.filterwarnings("ignore")
print("✅ Libraries ready.")

### 0.3. API Client Initialization (using Kaggle Secrets)

In [ ]:
client = None

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    client = genai.Client(api_key=GOOGLE_API_KEY)
    print("✅ GenAI client initialized.")
except NameError:
    print("❌ UserSecretsClient not found. Are you running outside Kaggle?")
except Exception as e:
    print(f"❌ Failed to init GenAI client: {e}")

if client is None:
    print("⚠️ GenAI client is unavailable. API calls will fail.")
    # raise RuntimeError("GenAI client init failed.")  # Uncomment if critical

### 0.4. Global Configuration

In [ ]:
# Model and style configuration
MODEL_NAME = "gemini-2.0-flash"  # Recommended: "gemini-2.0-flash", "gemini-1.5-pro-latest"
GENERATION_STYLE = "insightful"  # E.g., "engaging", "technical", "professional", etc.

print(f"✅ Model: {MODEL_NAME}")
print(f"🧠 Style: {GENERATION_STYLE}")

### 0.5. Define Output Data Structure (VideoMetadata TypedDict)

In [ ]:
from typing_extensions import TypedDict  # Use `typing` if Python >= 3.8

class VideoMetadata(TypedDict):
    topic_name: str
    title: str
    thumbnail_prompt: str
    description: str
    hashtags: list[str]
    tags: list[str]
    chapters: str

DEFAULT_METADATA: VideoMetadata = {
    "topic_name": "",
    "title": "",
    "thumbnail_prompt": "",
    "description": "",
    "hashtags": [],
    "tags": [],
    "chapters": ""
}

print("✅ VideoMetadata structure and defaults defined.")

# ==========================================
# 1. PROJECT OVERVIEW & USE CASE
# ==========================================

# 🧠 1. Introduction & Project Overview

## 🎯 Objective
In this project, we develop a **GenAI-powered agent** for YouTube creators. It generates complete metadata (title, topic, description, tags, chapters, thumbnail prompt) based on:
- A set of input ideas (**Scenario 1** – for upcoming livestream),
- or a full transcript (**Scenario 2** – for uploaded videos).

## 🤖 Key Features
This agent leverages **Gemini** models to:
- Generate structured, high-quality YouTube metadata;
- Ground responses using the built-in **Google Search tool**;
- Handle long transcripts via **Gemini's long context window**;
- Generate text prompts suitable for AI image generators.

## ✅ GenAI Capabilities Demonstrated
This notebook demonstrates **at least 5** distinct GenAI features relevant to the Capstone project:
- ✅ Structured Output / JSON Mode / Controlled Generation
- ✅ Grounding (via Google Search Tool)
- ✅ Retrieval Augmented Generation (RAG) concepts (using search results to inform generation)
- ✅ Long Context Window (handling full transcripts)
- ✅ Few-shot Prompting (using examples in prompts)

## 📊 Project Scenarios

| Scenario | Input | Output | Use Case |
|----------|--------|--------|----------|
| 1️⃣ Pre-Stream | Topic + Theses | Title, Description, Tags, Thumbnail Prompt | Planning metadata for an upcoming livestream |
| 2️⃣ Post-Video | Full Transcript | Title, Description, Tags, Chapters, Thumbnail Prompt | Generating/enhancing metadata for an existing video |

# ==========================================
# 2. UTILITY FUNCTIONS & HELPERS
# ==========================================

### 2.1. Google Trends Fetcher (Optional Enhancement for Tags)

In [ ]:
def get_google_trends_for_query(query: str, region: str = "US", timeframe: str = "now 7-d") -> list[str]:
    """
    Fetches top related search queries from Google Trends using pytrends.

    Args:
        query (str): The search term to analyze.
        region (str): Geographical region code (default is 'US').
        timeframe (str): Time window for the trend data (default is 'now 7-d').

    Returns:
        list[str]: A list of up to 5 related trending queries. Returns an empty list if unavailable.
    """
    if 'pytrends_client' not in globals() or pytrends_client is None:
        return []

    try:
        pytrends_client.build_payload(kw_list=[query], geo=region, timeframe=timeframe)
        related_queries = pytrends_client.related_queries()
        top_df = related_queries.get(query, {}).get("top")

        if isinstance(top_df, pd.DataFrame) and not top_df.empty:
            return top_df["query"].head(5).tolist()

        return []
    except Exception:
        return []

print("✅ Google Trends utility function ready.")

### 2.2. Metadata Rendering Function

In [ ]:
def render_metadata_md(metadata: VideoMetadata):
    """Render the video metadata dictionary as a formatted Markdown block."""
    if not isinstance(metadata, dict):
        return

    title = metadata.get("title", "N/A")
    topic = metadata.get("topic_name", "N/A")
    description = metadata.get("description", "N/A")
    chapters_raw = metadata.get("chapters", "")
    hashtags_list = metadata.get("hashtags", [])
    tags_list = metadata.get("tags", [])
    thumbnail_prompt = metadata.get("thumbnail_prompt", "N/A")

    hashtags_md = " ".join(
        list(dict.fromkeys(f"#{tag.strip().lstrip('#').split()[0].lower()}" for tag in hashtags_list if isinstance(tag, str) and tag.strip()))
    ) if hashtags_list else "None"

    tags_md = ", ".join(
        list(dict.fromkeys(str(t).strip() for t in tags_list if isinstance(t, str) and str(t).strip()))
    ) if tags_list else "None"

    chapters_md = "None"
    if chapters_raw and isinstance(chapters_raw, str):
        chapter_lines = [line.strip() for line in chapters_raw.strip().splitlines() if line.strip().startswith("➤")]
        if chapter_lines:
            chapters_md = "\n" + "  \n".join(chapter_lines)

    md_output = f"""
🎥 **Title:**
{title}

📌 **Topic:**
{topic}

📝 **Description:**
{description}

🎬 **Chapters:**
{chapters_md}

🏷️ **Hashtags:**
{hashtags_md}

🔍 **SEO Tags:**
{tags_md}

🖼️ **Thumbnail Prompt:**  
{thumbnail_prompt}
"""

    try:
        display(Markdown(md_output))
    except Exception:
        pass

print("✅ Metadata renderer ready.")

# ==========================================
# 3. CORE METADATA GENERATION COMPONENTS
# ==========================================
# This section contains the core building blocks:
# - Prompt templates defining instructions for the LLM.
# - Python functions that call the LLM with these prompts to generate specific metadata parts.

### 3.1. Topic & Title Generation

#### 3.1.A. Prompt Template (for Text Inputs)

In [ ]:
TOPIC_TITLE_PROMPT_FROM_INPUTS = """
You are a YouTube metadata assistant. Your task is to generate a clear topic and a catchy title for a YouTube video based on the provided user input (initial topic/idea) and keywords/theses.

Generate TWO lines:
1. A short and descriptive topic summarizing what the video is about.
   - Max 100 characters.
2. A concise and engaging YouTube-style title (preferably under 70 characters).

Format:
<Generated Topic>
<Generated Title>

NO explanations, no labels, no extra text.

---
User Input:
{topic}

Theses / Keywords:
{theses}
---
"""

print("✅ Topic & Title prompt (from user input) ready.")

#### 3.1.B. Function (for Text Inputs)

In [ ]:
def generate_topic_title_from_inputs(topic: str, theses: str) -> tuple[str, str]:
    """
    Generates a topic and a catchy title using Gemini based on user-provided topic and theses.

    Args:
        topic: The user's initial idea.
        theses: Supporting points or keywords.

    Returns:
        (topic_name, title) — both strings. Returns ("", "") on failure.
    """
    if 'client' not in globals() or client is None:
        print("❌ GenAI client not initialized.")
        return "", ""
    if 'MODEL_NAME' not in globals():
        print("❌ MODEL_NAME is not defined.")
        return "", ""
    if not topic or not theses:
        print("⚠️ Missing topic or theses input.")
        return "", ""

    try:
        prompt = TOPIC_TITLE_PROMPT_FROM_INPUTS.format(topic=topic, theses=theses)

        response = client.models.generate_content(
            model=MODEL_NAME,
            contents=[prompt]
        )

        text = response.text.strip()
        lines = text.split("\n", 1)

        if len(lines) == 2:
            topic_name = lines[0].strip().removeprefix("- Line 1:").strip()
            title = lines[1].strip().removeprefix("- Line 2:").strip()
            return topic_name, title
        else:
            print("⚠️ Unexpected output format.")
            return "", text if len(text) < 100 else ""

    except Exception as e:
        print(f"❌ Generation error: {e}")
        return "", ""

print("✅ Topic/title generation function ready.")

#### 3.1.C. Prompt Template (for Transcript Analysis)

In [ ]:
TOPIC_TITLE_PROMPT_FROM_TRANSCRIPT = """
You are a YouTube metadata assistant. Analyze the **full transcript** of a video to understand its core theme.

Generate exactly TWO lines:
1. A concise **topic** summarizing the video's core subject.
   - Max 100 characters.
   - A short sentence or phrase.
2. A **catchy YouTube-style title** (under 70 characters preferred).
   - Designed to attract viewers interested in the topic.

📌 Format (strict):
<topic>
<title>

❌ Do NOT include any labels, numbers, colons, or explanations.

✅ Example:
Understanding Large Language Models in Plain English
LLMs Explained: How ChatGPT Really Works

---
Full Transcript:
{transcript_text}
---

Generate the output now:
"""

print("✅ Prompt template `TOPIC_TITLE_PROMPT_FROM_TRANSCRIPT` defined.")

#### 3.1.D. Function (for Transcript Analysis)

In [ ]:
def generate_topic_title_from_transcript(transcript_text: str) -> tuple[str, str]:
    """
    Generates a topic and title by analyzing a full video transcript using the Gemini model.

    Args:
        transcript_text (str): The complete video transcript.

    Returns:
        tuple[str, str]: A pair of strings (topic, title), or ("", "") on failure.
    """
    if 'client' not in globals() or client is None:
        print("❌ GenAI client is not initialized.")
        return "", ""

    if 'MODEL_NAME' not in globals():
        print("❌ MODEL_NAME is not defined.")
        return "", ""

    if not transcript_text:
        print("⚠️ Empty transcript. Skipping topic/title generation.")
        return "", ""

    try:
        prompt = TOPIC_TITLE_PROMPT_FROM_TRANSCRIPT.format(transcript_text=transcript_text)
        response = client.models.generate_content(model=MODEL_NAME, contents=[prompt])
        text = response.text.strip()
        lines = text.split("\n", 1)

        if len(lines) == 2:
            topic = lines[0].strip()
            title = lines[1].strip()

            # Handle accidental labeling
            if topic.lower().startswith("- line 1:"):
                topic = topic.split(":", 1)[-1].strip()
            if title.lower().startswith("- line 2:"):
                title = title.split(":", 1)[-1].strip()

            return topic, title
        else:
            return "", text if len(text) < 100 else ""

    except Exception as e:
        print(f"❌ Error during topic/title generation from transcript: {e}")
        return "", ""

print("✅ Function `generate_topic_title_from_transcript` defined.")

### 3.2. Description Generation (with Grounding)

#### 3.2.A. Prompt Template

In [ ]:
# Prompt template for generating a YouTube video description using topic, title,
# content (theses or transcript), and grounded Google Search results.

DESCRIPTION_PROMPT_TEMPLATE = """
You are a creative YouTube content writer. Your task is to write a short, compelling video description based on the video topic, title, main talking points (or transcript), and relevant Google Search results for context. Use a '{style}' tone.

Guidelines:
- Begin with a short, {style} introduction.
- If given theses, summarize them using a concise bulleted list.
- If given a transcript, summarize the key ideas in flowing text (no bullets).
- Use search results as context where relevant, but prioritize the main content.
- Avoid generic or overly promotional language.
- Write in English.
- Output only the description text (no headings, labels, or extra explanation).

---
🎯 Topic: {topic_name}
🎬 Title: {title}
📌 Main Points / Transcript Snippet:
{main_points_or_transcript}

🔍 Background Search Results:
{search_results}
---

Write the final video description below:
"""

print("✅ Prompt template `DESCRIPTION_PROMPT_TEMPLATE` defined.")

#### 3.2.B. Function

In [ ]:
def generate_description_with_search(topic_name: str, title: str, main_points_or_transcript: str) -> str:
    """
    Generates a YouTube video description using the Gemini model,
    grounded with Google Search results for context.

    Args:
        topic_name: The main topic of the video.
        title: The title of the video.
        main_points_or_transcript: Either key points (Scenario 1) or full transcript (Scenario 2).

    Returns:
        A generated video description string, or an empty string on failure.
    """
    if 'client' not in globals() or client is None:
        return ""
    if 'MODEL_NAME' not in globals() or 'GENERATION_STYLE' not in globals():
        return ""
    if not topic_name or not title or not main_points_or_transcript:
        return ""

    # --- Step 1: Google Search Tool (Grounding) ---
    search_text = "No background search information available."
    try:
        from google.genai import types
        snippet = main_points_or_transcript[:200]
        search_query = f"Latest insights about: {topic_name} - {title}. Related points: {snippet}"
        config_with_search = types.GenerateContentConfig(
            tools=[types.Tool(google_search=types.GoogleSearch())]
        )
        search_response = client.models.generate_content(
            model=MODEL_NAME,
            contents=[search_query],
            config=config_with_search
        )
        search_text = search_response.candidates[0].content.parts[0].text.strip()
    except Exception:
        pass  # Proceed without search context if error occurs

    # --- Step 2: Generate Description ---
    try:
        prompt = DESCRIPTION_PROMPT_TEMPLATE.format(
            topic_name=topic_name.strip(),
            title=title.strip(),
            main_points_or_transcript=main_points_or_transcript.strip(),
            search_results=search_text,
            style=GENERATION_STYLE
        )
        response = client.models.generate_content(
            model=MODEL_NAME,
            contents=[prompt]
        )
        return response.text.strip()
    except Exception:
        return ""

print("✅ Function `generate_description_with_search` defined.")

### 3.3. Tags & Keywords Generation (JSON Output)

#### 3.3.A. Prompt Template

In [ ]:
TAGS_PROMPT_TEMPLATE = """
You are a YouTube SEO expert. Based on the video topic and description below, generate optimized metadata.

Generate:
- A list of up to 10 relevant **hashtags**:
  - Start each with '#'
  - Lowercase only
  - Use single words or short phrases (e.g., #generativeai)
- A list of **SEO keyword tags**:
  - Lowercase
  - Single words or 2–4 word phrases
  - Avoid generic words like "cool", "video", etc.

Respond with a **valid JSON object** with two keys:
- "hashtags": [list of strings]
- "tags": [list of strings]

Do not include any explanations or text outside the JSON.

---
🎯 Topic: {topic_name}
📝 Description:
{description}
---

Example JSON output:
```json
{{
  "hashtags": ["#generativeai", "#youtubeai", "#videotools"],
  "tags": ["ai for content creators", "youtube metadata generation", "seo for youtube"]
}}
"""
print("✅ Prompt template `TAGS_PROMPT_TEMPLATE` defined.")

#### 3.3.B. Function

In [ ]:
def generate_tags_and_keywords(topic_name: str, description: str, use_trends: bool = True) -> tuple[list[str], list[str]]:
    """
    Generates hashtags and SEO tags for YouTube videos using the Gemini model,
    optionally enhanced with Google Trends data.
    """
    if 'client' not in globals() or client is None:
        return [], []
    if 'MODEL_NAME' not in globals():
        return [], []
    if not topic_name or not description:
        return [], []

    hashtags, tags = [], []

    try:
        prompt = TAGS_PROMPT_TEMPLATE.format(
            topic_name=topic_name.strip(),
            description=description.strip()
        )

        response = client.models.generate_content(
            model=MODEL_NAME,
            contents=[prompt]
        )

        raw_text = response.text.strip()

        # Remove potential code fences
        if raw_text.startswith("```json"):
            raw_text = raw_text[7:].strip()
        if raw_text.startswith("```"):
            raw_text = raw_text[3:].strip()
        if raw_text.endswith("```"):
            raw_text = raw_text[:-3].strip()

        try:
            tags_json = json.loads(raw_text)
            hashtags = tags_json.get("hashtags", [])
            tags = tags_json.get("tags", [])
            if not isinstance(hashtags, list):
                hashtags = []
            if not isinstance(tags, list):
                tags = []
        except json.JSONDecodeError:
            return [], []

    except Exception:
        return [], []

    # --- Optional: Enhance with Google Trends ---
    if use_trends and 'get_google_trends_for_query' in globals():
        try:
            trends = get_google_trends_for_query(topic_name)
            if isinstance(trends, list):
                for t in trends:
                    t_clean = t.lower().strip()
                    if t_clean and t_clean not in tags:
                        tags.append(t_clean)
                    h_tag = f"#{t_clean.replace(' ', '')}"
                    if h_tag != "#" and h_tag not in hashtags:
                        hashtags.append(h_tag)
        except Exception:
            pass

    # --- Final cleanup: ensure all hashtags are clean, start with #, no duplicates ---
    final_hashtags = []
    for h in hashtags:
        if isinstance(h, str):
            clean = h.strip().lstrip('#').replace(' ', '')
            if clean:
                h_tag = f"#{clean}"
                if h_tag not in final_hashtags:
                    final_hashtags.append(h_tag)

    final_tags = []
    for t in tags:
        if isinstance(t, str):
            clean = t.strip()
            if clean and clean not in final_tags:
                final_tags.append(clean)

    return final_hashtags, final_tags
print("✅ Function `generate_tags_and_keywords` defined.")

### 3.4. Chapter Generation (from Transcript)
# Note: This component is used only in Scenario 2 (metadata from transcript).

#### 3.4.A. Prompt Template

In [ ]:
CHAPTERS_PROMPT = """
You are a YouTube assistant. Analyze the full transcript below and identify all major topic shifts or sections.

Your task:
- Generate a list of clear and concise chapter titles.
- Each title must be paired with a precise timestamp in the format HH:MM:SS or MM:SS.
- Chapters must reflect the actual structure and content flow of the video.

Strict formatting rules:
- Plain text output ONLY. No markdown, HTML, JSON, or code blocks.
- DO NOT include any intro, explanation, or summary.
- Each line MUST follow this format:
  ➤ HH:MM:SS Chapter Title Here
- Start each chapter on a new line.
- Begin with the very first chapter (e.g., ➤ 00:00 Introduction).
- Use the exact symbol "➤" (U+27A4) followed by a space.
- No numbering (1., 2.), no speaker names, no bullet points.

---
Transcript:
{transcript}
---

Now generate the chapter list only, following the format exactly:
"""

print("✅ Prompt template `CHAPTERS_PROMPT` defined.")

#### 3.4.B. Function

In [ ]:
def generate_chapters_from_transcript(transcript_text: str) -> str:
    """
    Generates YouTube video chapters by analyzing the full transcript using the Gemini model.

    Args:
        transcript_text: The full transcript text of the video.

    Returns:
        A string with formatted chapters (one per line, starting with ➤ HH:MM:SS Title),
        or an empty string if generation fails.
    """
    if 'client' not in globals() or client is None:
        print("❌ GenAI client not initialized.")
        return ""
    if 'MODEL_NAME' not in globals():
        print("❌ MODEL_NAME not defined.")
        return ""
    if not transcript_text:
        print("⚠️ Empty transcript provided.")
        return ""

    print(f"🎬 Generating chapters (transcript length: {len(transcript_text)} chars)...")

    try:
        prompt = CHAPTERS_PROMPT.format(transcript=transcript_text)
        response = client.models.generate_content(
            model=MODEL_NAME,
            contents=prompt
        )

        chapters_output = response.text.strip()
        if chapters_output:
            num_lines = sum(1 for line in chapters_output.splitlines() if line.startswith("➤"))
            print(f"✅ Chapters generated: {num_lines} lines.")
        else:
            print("⚠️ Chapter generation returned empty output.")

        return chapters_output

    except Exception as e:
        print(f"❌ Error generating chapters: {e}")
        return ""

print("✅ Function `generate_chapters_from_transcript` defined.")

### 3.5. Thumbnail Prompt Generation

#### 3.5.A. Prompt Template

In [ ]:
THUMBNAIL_PROMPT_TEMPLATE = """
You are a creative visual director crafting image prompts for AI generators (e.g., Midjourney, DALL·E, Stable Diffusion).
Based on the YouTube video's topic, title, and main points, generate a vivid and relevant prompt for a thumbnail image.

Visual style: **{style}**

Requirements:
- Focus only on visual elements: setting, composition, key subjects, mood, colors, lighting.
- No text or captions in the prompt (avoid "Text: ...").
- Avoid real-world names; use symbolic/general descriptors.
- Output only the prompt text — no explanations or labels.

---
🎯 Topic: {topic_name}
🎬 Title: {title}
📌 Summary / Key Points:
{main_points_or_transcript}
---

Now generate the thumbnail image prompt:
"""

print("✅ Prompt template `THUMBNAIL_PROMPT_TEMPLATE` defined.")

#### 3.5.B. Function

In [ ]:
def generate_thumbnail_prompt(topic_name: str, title: str, main_points_or_transcript: str) -> str:
    """
    Generates a visual prompt for an AI image generator to create a YouTube thumbnail.

    Args:
        topic_name: Video topic.
        title: Video title.
        main_points_or_transcript: Key points or transcript summary.

    Returns:
        Thumbnail prompt string, or empty string if generation fails.
    """
    if not all([client, MODEL_NAME, GENERATION_STYLE, topic_name, title, main_points_or_transcript]):
        print("❌ Missing required inputs or client not initialized.")
        return ""

    print(f"🎨 Generating thumbnail prompt ({GENERATION_STYLE})...")
    try:
        prompt = THUMBNAIL_PROMPT_TEMPLATE.format(
            topic_name=topic_name.strip(),
            title=title.strip(),
            main_points_or_transcript=main_points_or_transcript.strip(),
            style=GENERATION_STYLE
        )

        response = client.models.generate_content(
            model=MODEL_NAME,
            contents=[prompt]
        )

        text = response.text.strip()
        if not text:
            print("⚠️ Empty response.")
            return ""

        print("✅ Thumbnail prompt generated.")
        return text

    except Exception as e:
        print(f"❌ Error generating thumbnail prompt: {e}")
        return ""
print("✅ Function `generate_thumbnail_prompt` defined.")

# ==========================================
# 4. SCENARIO ORCHESTRATORS
# ==========================================
# This section contains functions that orchestrate the calls to the core
# generation components (defined in Section 3) to implement the two main
# use cases of the agent.

### 4.1. Scenario 1: Generate Metadata from Text Inputs
# Use case: Generate initial metadata for a planned video/livestream
# based on a topic idea and key talking points (theses).

#### 4.1.A. Orchestrator Function

In [ ]:
def generate_metadata_from_text(topic: str, theses: str, use_trends_for_tags: bool = True) -> VideoMetadata:
    """
    Generates complete video metadata (except chapters) from input topic and theses.

    Args:
        topic: User's topic idea.
        theses: Key points or concepts for the video.
        use_trends_for_tags: Whether to enhance tags using Google Trends.

    Returns:
        A VideoMetadata dictionary containing all generated metadata fields.
    """
    print("\n🎬 Starting Metadata Generation from Text Inputs...")

    metadata: VideoMetadata = DEFAULT_METADATA.copy()
    metadata['chapters'] = ""  # Not generated in this scenario

    try:
        topic_gen, title_gen = generate_topic_title_from_inputs(topic, theses)
        metadata['topic_name'] = topic_gen or topic
        metadata['title'] = title_gen or "Title Generation Failed"
    except Exception as e:
        print(f"⚠️ Topic/Title generation error: {e}")
        metadata['topic_name'] = topic
        metadata['title'] = "Title Generation Error"

    try:
        description = generate_description_with_search(
            topic_name=metadata['topic_name'],
            title=metadata['title'],
            main_points_or_transcript=theses
        )
        metadata['description'] = description or ""
    except Exception as e:
        print(f"⚠️ Description generation error: {e}")

    try:
        hashtags, tags = generate_tags_and_keywords(
            topic_name=metadata['topic_name'],
            description=metadata['description'],
            use_trends=use_trends_for_tags
        )
        metadata['hashtags'] = hashtags
        metadata['tags'] = tags
    except Exception as e:
        print(f"⚠️ Tags/Keywords generation error: {e}")

    try:
        thumbnail_prompt = generate_thumbnail_prompt(
            topic_name=metadata['topic_name'],
            title=metadata['title'],
            main_points_or_transcript=theses
        )
        metadata['thumbnail_prompt'] = thumbnail_prompt or ""
    except Exception as e:
        print(f"⚠️ Thumbnail prompt generation error: {e}")

    print("✅ Scenario 1 Metadata Generation Complete.")
    return metadata

print("✅ Orchestrator function `generate_metadata_from_text` defined.")

### 4.2. Scenario 2: Generate Metadata from Full Transcript
# Use case: Generate comprehensive metadata, including chapters,
# for an existing video using its full text transcript.

#### 4.2.A. Helper: Load Transcript File

In [ ]:
def load_transcript(file_path: str) -> str:
    """
    Loads the video transcript from a given file path.

    Args:
        file_path: Path to the .txt file containing the transcript.

    Returns:
        The transcript as a string, or an empty string if loading fails.
    """
    if not os.path.exists(file_path):
        print(f"❌ Transcript file not found: {file_path}")
        return ""
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            text = f.read().strip()
        if not text:
            print("⚠️ Transcript file is empty.")
            return ""
        print(f"✅ Transcript loaded ({len(text)} characters).")
        return text
    except Exception as e:
        print(f"❌ Error reading transcript: {e}")
        return ""

print("✅ Helper function `load_transcript` defined.")

#### 4.2.B. Orchestrator Function

In [ ]:
def generate_metadata_from_transcript(transcript_text: str, use_trends_for_tags: bool = True) -> VideoMetadata:
    """
    Generates full metadata for a YouTube video based on its transcript (Scenario 2).

    Args:
        transcript_text: Full video transcript as a string.
        use_trends_for_tags: Whether to enhance tags using Google Trends.

    Returns:
        A dictionary of generated metadata conforming to VideoMetadata structure.
    """
    if not transcript_text:
        return DEFAULT_METADATA.copy()

    metadata: VideoMetadata = DEFAULT_METADATA.copy()

    try:
        topic, title = generate_topic_title_from_transcript(transcript_text)
        metadata['topic_name'] = topic or "Topic Generation Failed"
        metadata['title'] = title or "Title Generation Failed"
    except:
        metadata['topic_name'] = "Topic Generation Error"
        metadata['title'] = "Title Generation Error"

    try:
        description = generate_description_with_search(
            topic_name=metadata['topic_name'],
            title=metadata['title'],
            main_points_or_transcript=transcript_text
        )
        metadata['description'] = description
    except:
        metadata['description'] = ""

    try:
        chapters = generate_chapters_from_transcript(transcript_text)
        metadata['chapters'] = chapters or ""
    except:
        metadata['chapters'] = ""

    try:
        hashtags, tags = generate_tags_and_keywords(
            topic_name=metadata['topic_name'],
            description=metadata['description'],
            use_trends=use_trends_for_tags
        )
        metadata['hashtags'] = hashtags
        metadata['tags'] = tags
    except:
        metadata['hashtags'] = []
        metadata['tags'] = []

    try:
        thumbnail = generate_thumbnail_prompt(
            topic_name=metadata['topic_name'],
            title=metadata['title'],
            main_points_or_transcript=transcript_text
        )
        metadata['thumbnail_prompt'] = thumbnail
    except:
        metadata['thumbnail_prompt'] = ""

    print("✅ Scenario 2 Metadata Generation Complete.")
    return metadata

print("✅ Orchestrator function `generate_metadata_from_transcript` defined.")

# ==========================================
# 5. DEMONSTRATION
# ==========================================
# This section demonstrates how to use the orchestrator functions
# defined in Section 4 for both Scenario 1 and Scenario 2.

### 5.1. Demo: Scenario 1 (Text Inputs)
# Example of generating metadata based on an initial topic and theses.

#### 5.1.A. Define Example Inputs

In [ ]:
# 📌 Define inputs for Scenario 1 (metadata from topic + theses)
demo_topic_s1 = "AI Impact on Creative Jobs"
demo_theses_s1 = """
- Explore the current state of AI tools (image generators, music AI, writing assistants) in creative fields.
- Discuss potential job displacement vs. job augmentation (new roles, required skills).
- Analyze the need for adaptation strategies for creative professionals.
- Briefly touch upon ethical considerations and the future outlook for human creativity.
"""
print("✅ Scenario 1 demo inputs defined.")

#### 5.1.B. Run Generation

In [ ]:
# 🔄 Run metadata generation for Scenario 1 (from topic + theses)
s1_metadata_result = generate_metadata_from_text(
    topic=demo_topic_s1,
    theses=demo_theses_s1,
    use_trends_for_tags=True
)

#### 5.1.C. Render Output

In [ ]:
# 🖼️ Display generated metadata for Scenario 1
if s1_metadata_result:
    render_metadata_md(s1_metadata_result)
else:
    print("⚠️ Metadata not available.")

### 5.2. Demo: Scenario 2 (Transcript)
# Example of generating metadata based on a full video transcript file.

#### 5.2.A. Define Transcript Path & Load Data (from Kaggle Input)

In [ ]:
# 📄 Load transcript file (Scenario 2)
transcript_file_path_s2 = "/kaggle/input/test3-3/test3.txt"  # Ensure this file is available via Kaggle input dataset

# Load using previously defined helper function
raw_transcript_s2 = load_transcript(transcript_file_path_s2) if callable(globals().get("load_transcript", None)) else ""

# Final check
if not raw_transcript_s2:
    print("⚠️ Transcript not loaded. Scenario 2 will be skipped.")
else:
    print("✅ Transcript loaded successfully. Proceeding to Scenario 2.")

#### 5.2.B. Run Generation

In [ ]:
# ▶️ Run Scenario 2 (if transcript is available)
s2_metadata_result = None
if raw_transcript_s2:
    s2_metadata_result = generate_metadata_from_transcript(
        transcript_text=raw_transcript_s2,
        use_trends_for_tags=True
    )

#### 5.2.C. Render Output

In [ ]:
# ▶️ Display metadata generated from transcript (Scenario 2)
print("\n--- Scenario 2: Generated Metadata ---") 
if s2_metadata_result: 
    render_metadata_md(s2_metadata_result) 
elif not raw_transcript_s2: 
    print("ℹ️ Skipped: No transcript loaded.") 
else: 
    print("⚠️ Metadata generation failed. See logs above.")

# ==========================================
# 6. CONCLUSION & NEXT STEPS
# ==========================================

## Conclusion

This notebook successfully demonstrates a GenAI-powered agent capable of automating YouTube metadata generation. By leveraging Gemini models, grounding via Google Search, and processing potentially long transcripts, the agent can produce relevant titles, topics, descriptions, tags, chapters, and thumbnail prompts for two common scenarios: planning upcoming streams and processing existing video transcripts.

This significantly reduces the manual effort involved in preparing video content for publication, allowing creators (like myself!) to focus more on the creative aspects.

## Limitations and Future Work

As a first version developed within the scope of the Kaggle Capstone project, this agent certainly has room for improvement and refinement. Some current limitations include:

* **Output Variability:** The quality and style of generated metadata depend heavily on the model's interpretation and can sometimes require manual tweaking.
* **Input Dependency:** The quality of generated metadata (especially for Scenario 2) is dependent on the accuracy and completeness of the input transcript.
* **Error Handling:** While basic error handling is implemented, more sophisticated checks and recovery mechanisms could be added.
* **Static Configuration:** Model name, style, and other parameters are currently set globally.

Based on the ideas for future development, potential next steps could include:

* **Google Sheets Integration:** Connect the agent to Google Sheets for batch processing of video ideas or storing generated metadata.
* **YouTube API Integration:** Directly upload or update video metadata to YouTube via its API.
* **Speaker/Guest Database:** Integrate a database or lookup system for regular speakers/guests to automatically include their information and social links in descriptions.
* **Output Evaluation:** Implement automated checks or use another LLM to evaluate the quality, relevance, and SEO potential of the generated metadata.
* **Cost & Usage Tracking:** Add mechanisms to estimate and track API usage costs for generation.
* **Enhanced Prompt Engineering:** Further refine prompts for even more consistent and controllable outputs.
* **User Interface:** Develop a simple UI (e.g., using Gradio or Streamlit) for easier interaction.

This project serves as a solid foundation, and further development could turn it into an even more powerful tool for YouTube creators. Feedback, critical assessments, and suggestions for improvement are highly welcome!